In [ ]:
import numpy as np
import os

class EEGStreamHandler:
    """
    Class for reading and writing streams of NumPy arrays to/from binary files.
    Handles two separate files: one for EEG data and one for timestamps.
    
    EEG data: np.float32, shape (12, 4)
    Timestamps: np.float64, shape (12, 1)
    """
    
    def __init__(self, data_file_path, timestamp_file_path, mode='r'):
        """
        Initialize the EEG stream handler.
        
        Parameters:
        -----------
        data_file_path : str
            Path to the EEG data file
        timestamp_file_path : str
            Path to the timestamps file
        mode : str
            'r' for read, 'w' for write, 'a' for append
        """
        self.data_file_path = data_file_path
        self.timestamp_file_path = timestamp_file_path
        self.mode = mode
        
        # Define expected shapes and dtypes
        self.eeg_shape = (12, 4)
        self.eeg_dtype = np.float32
        self.timestamp_shape = (12,)
        self.timestamp_dtype = np.float64
        
        # Calculate sizes for verification and seeking
        self.eeg_frame_size = np.prod(self.eeg_shape) * np.dtype(self.eeg_dtype).itemsize
        self.timestamp_frame_size = np.prod(self.timestamp_shape) * np.dtype(self.timestamp_dtype).itemsize
        print(np.dtype(self.timestamp_dtype).itemsize)
        # Open file handlers based on mode
        if mode == 'r':
            self._open_for_reading()
        elif mode in ('w', 'a'):
            self._open_for_writing(mode)
        else:
            raise ValueError(f"Invalid mode: {mode}. Use 'r', 'w', or 'a'.")
    
    def _open_for_reading(self):
        """Open files for reading."""
        if not os.path.exists(self.data_file_path) or not os.path.exists(self.timestamp_file_path):
            raise FileNotFoundError("Data file or timestamp file not found.")
        
        self.data_file = open(self.data_file_path, 'rb')
        self.timestamp_file = open(self.timestamp_file_path, 'rb')
        
        # Get file sizes for frame count calculation
        self.data_file.seek(0, 2)  # Seek to end
        data_file_size = self.data_file.tell()
        self.data_file.seek(0)  # Reset to beginning
        
        self.timestamp_file.seek(0, 2)  # Seek to end
        timestamp_file_size = self.timestamp_file.tell()
        self.timestamp_file.seek(0)  # Reset to beginning
        
        # Calculate total frames
        self.total_data_frames = data_file_size // self.eeg_frame_size
        self.total_timestamp_frames = timestamp_file_size // self.timestamp_frame_size
        
        # Verify consistency
        if self.total_data_frames != self.total_timestamp_frames:
            print(f"Warning: Mismatch between data frames ({self.total_data_frames}) "
                  f"and timestamp frames ({self.total_timestamp_frames}).")
    
    def _open_for_writing(self, mode):
        """Open files for writing or appending."""
        self.data_file = open(self.data_file_path, mode + 'b')
        self.timestamp_file = open(self.timestamp_file_path, mode + 'b')
    
    def write_frame(self, eeg_data, timestamp):
        """
        Write a single frame of EEG data and timestamp.
        
        Parameters:
        -----------
        eeg_data : np.ndarray
            EEG data array with shape (12, 4) and dtype np.float32
        timestamp : np.ndarray
            Timestamp array with shape (12, 1) and dtype np.float64
        
        Returns:
        --------
        bool
            True if successful, False otherwise
        """
        if self.mode not in ('w', 'a'):
            raise IOError("File not opened for writing.")
        
        # Validate inputs
        if not isinstance(eeg_data, np.ndarray) or eeg_data.shape != self.eeg_shape or eeg_data.dtype != self.eeg_dtype:
            raise ValueError(f"EEG data must be a numpy array with shape {self.eeg_shape} and dtype {self.eeg_dtype}")
        
        if not isinstance(timestamp, np.ndarray) or timestamp.shape != self.timestamp_shape or timestamp.dtype != self.timestamp_dtype:
            raise ValueError(f"Timestamp must be a numpy array with shape {self.timestamp_shape} and dtype {self.timestamp_dtype}")
        
        # Write data
        self.data_file.write(eeg_data.tobytes())
        self.timestamp_file.write(timestamp.tobytes())
        
        # Flush to ensure data is written
        self.data_file.flush()
        self.timestamp_file.flush()
        
        return True
    
    def read_frame(self, frame_index=None):
        """
        Read a single frame of EEG data and its corresponding timestamp.
        
        Parameters:
        -----------
        frame_index : int, optional
            Index of the frame to read. If None, reads the next frame.
        
        Returns:
        --------
        tuple
            (eeg_data, timestamp) if successful, (None, None) if EOF
        """
        if self.mode != 'r':
            raise IOError("File not opened for reading.")
        
        # Seek to specific frame if requested
        if frame_index is not None:
            if frame_index < 0 or frame_index >= self.total_data_frames:
                raise IndexError(f"Frame index {frame_index} out of range (0-{self.total_data_frames-1})")
            
            self.data_file.seek(frame_index * self.eeg_frame_size)
            self.timestamp_file.seek(frame_index * self.timestamp_frame_size)
        
        # Read EEG data
        eeg_bytes = self.data_file.read(self.eeg_frame_size)
        if len(eeg_bytes) < self.eeg_frame_size:
            return None, None  # EOF reached
        
        # Read timestamp
        timestamp_bytes = self.timestamp_file.read(self.timestamp_frame_size)
        if len(timestamp_bytes) < self.timestamp_frame_size:
            return None, None  # EOF reached
        
        # Convert to numpy arrays
        eeg_data = np.frombuffer(eeg_bytes, dtype=self.eeg_dtype).reshape(self.eeg_shape)
        timestamp = np.frombuffer(timestamp_bytes, dtype=self.timestamp_dtype).reshape(self.timestamp_shape)
        
        return eeg_data, timestamp
    
    def read_all_frames(self):
        """
        Generator that yields frames one at a time from the beginning of the files.
        
        Yields:
        -------
        tuple
            (eeg_data, timestamp) for each frame
        """
        if self.mode != 'r':
            raise IOError("File not opened for reading.")
        
        # Seek to beginning
        self.data_file.seek(0)
        self.timestamp_file.seek(0)
        
        # Yield frames one by one
        for _ in range(self.total_data_frames):
            eeg_data, timestamp = self.read_frame()
            if eeg_data is None or timestamp is None:
                break
            
            yield eeg_data, timestamp
    
    def close(self):
        """Close all file handlers."""
        if hasattr(self, 'data_file') and self.data_file:
            self.data_file.close()
        
        if hasattr(self, 'timestamp_file') and self.timestamp_file:
            self.timestamp_file.close()
    
    def __enter__(self):
        """Context manager entry."""
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit."""
        self.close()

In [ ]:
with EEGStreamHandler('data/02-27_00-16-35/EEG_data.bin', 'data/02-27_00-16-35/EEG_timestamp.bin', mode='w') as eeg_stream_handler:
    for i in range(3):
        eeg_data = np.random.rand(2, 4).astype(np.float32)
        timestamp = np.random.rand(2).astype(np.float64)
        eeg_stream_handler.write_frame(eeg_data, timestamp)
        print(eeg_data)
        print(timestamp)

import time
with EEGStreamHandler('data/02-27_00-16-35/EEG_data.bin', 'data/02-27_00-16-35/EEG_timestamp.bin', mode='r') as eeg_stream_handler:
    for data, timestamps in eeg_stream_handler.read_all_frames():
        print(data)
        print(timestamps)
        time.sleep(3)